In [1]:
import numpy as np
from plotly import graph_objects as go
from plotly.subplots import make_subplots

import witss

## Basic iterated sums

In [2]:
a = np.random.normal(size=10_000)

itsum = witss.iss(a, "[1][1]")
itsum_norm = witss.iss(a, "[1][1]", normalize="sqrt")

In [3]:
fig = make_subplots(3, 1, shared_xaxes=True)
fig.add_trace(go.Scatter(
    y=a,
    mode="lines",
    name="Input",
), row=1, col=1)
fig.add_trace(go.Scatter(
    y=itsum,
    mode="lines",
    name="Iterated Sum",
), row=2, col=1)
fig.add_trace(go.Scatter(
    y=itsum_norm,
    mode="lines",
    name="Normalized Iterated Sum",
), row=3, col=1)
fig.update_layout(
    margin=dict(l=0,r=0,b=0,t=0),
)
fig.show()

## Compute the signature up to some total word weight


In [4]:
x = np.random.normal(size=(10, 20_000, 2))

iss = witss.iss(x, witss.words.up_to_weight(4, dim=2))

print("Shape of iss(x, [1][1][2]):",iss["[1][1][2]"].shape)
print("Shape of the signature up to depth 4:", iss.numpy().shape)

Shape of iss(x, [1][1][2]): (10, 20000)
Shape of the signature up to depth 4: (115, 10, 20000)


## Speed of computations

In [5]:
# 100 time series of length 10_000 with 2 dimensions
x = np.random.random((100, 10_000, 2))

#### Batches

In [6]:
%%timeit -n 10

witss.iss(x, witss.Word("[1][1][1][1]"), batches=1)

50.8 ms ± 4.91 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
%%timeit -n 10

witss.iss(x, witss.Word("[1][1][1][1]"), batches=250)

5.73 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


#### BagOfWords

In [8]:
bagofwords = witss.BagOfWords(*[i*"[1]" for i in range(1, 10)])
words = bagofwords.words()
print(bagofwords.explain())

0: [1][1][1][1][1][1][1][1][1] - computed
1: [1][1][1][1][1][1][1][1]    - prefix of 0
2: [1][1][1][1][1][1][1]       - prefix of 0
3: [1][1][1][1][1][1]          - prefix of 0
4: [1][1][1][1][1]             - prefix of 0
5: [1][1][1][1]                - prefix of 0
6: [1][1][1]                   - prefix of 0
7: [1][1]                      - prefix of 0
8: [1]                         - prefix of 0


In [9]:
%%timeit -n 10

witss.iss(x, bagofwords, batches=250)

40.5 ms ± 1.02 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit -n 10

[witss.iss(x, w, batches=250) for w in words]

49.6 ms ± 608 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
